# Steps to follow in order to get the selected interpration by using LLama:
1. run all data train and test with the new prompt with llama
2. interpretations = calculate_prior_probability(interpretations, w_llm=0.5, w_bscore =0.7)
3. interpretations = calculate_fit_quality(interpretations,w_rwp=1, w_score=0.5)
4. interpretations = calculate_posterior_probability_of_interpretation(interpretations)


Make sure LLM_interpretation_likelihood is updated with the LLama's answer & you store the final prior and fit_quality and posterior with different name, do not overwrite the excisting one. Then we can compare the selected interpretation(higher prob) by using Chatgpt vs LLama vs "lowset rwp valus"

In [7]:
def calculate_prior_probability(interpretations, w_llm=1, w_bscore=1):
    # Update each interpretation with a new field "prior_probability"
    for interpretation_name, interpretation in interpretations.items():
        interpretation["prior_probability_llama"] = (
           (interpretation["LLM_interpretation_likelihood_llama"]*w_llm + interpretation["balance_score"]*w_bscore)/(w_llm + w_bscore)
        )
    return interpretations

In [4]:
def calculate_fit_quality(interpretations, w_rwp=1, w_score=1):
    for key, interpretation in interpretations.items():
        interpretation["fit_quality"] = (interpretation["normalized_rwp"]*w_rwp + interpretation["normalized_score"]*w_score) / (w_rwp + w_score)
    return interpretations

In [3]:
def calculate_posterior_probability_of_interpretation(interpretations):
    """
    Calculate the posterior probabilities P(In | S) for all interpretations
    using balance_score and interpretation_importance.
    
    Parameters:
        interpretations (dict or list): A dictionary of interpretations or 
                                        a list containing a single dictionary.

    Returns:
        dict: Updated interpretations with added "posterior_probability" field.
    """
    # Extract dictionary if interpretations is a list
    if isinstance(interpretations, list) and len(interpretations) == 1:
        interpretations = interpretations[0]  # Extract the dictionary

    if not isinstance(interpretations, dict):
        raise ValueError("Interpretations should be a dictionary or a list containing a dictionary.")

     # Step 1: Compute unnormalized posterior = prior * fit_quality
    joint_probabilities = {}
    for name, interp in interpretations.items():
        joint = interp["prior_probability_llama"] * interp["fit_quality_llama"]
        interpretations[name]["unnormalized_posterior_llama"] = joint
        joint_probabilities[name] = joint

    # Step 2: Normalize to get posterior_probability
    total_joint = sum(joint_probabilities.values())
    for name in interpretations:
        if total_joint > 0:
            interpretations[name]["posterior_probability_llama"] = (
                interpretations[name]["unnormalized_posterior_llama"] / total_joint
            )
        else:
            interpretations[name]["posterior_probability_llama"] = 0.0

    return interpretations  

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import json
import os

def load_json(file_path): # Load JSON Data
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            try:
                return json.load(f)
            except json.JSONDecodeError:
                print("JSON decode error — file is empty or malformed.")
                return {}
    else:
        return {}  # Return empty dict if file doesn't exist


def update_json(data, key, value): # Modify/Add Data
    data[key] = value
    return data

def save_json(file_path, data): # Save JSON Data
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [ ]:
file_path = "Data/prompt2/interpretations_llm_v5.1_llama2.json"
json_file = load_json(file_path)
for sample in json_file:
    interpretations = json_file[sample]
    interpretations = calculate_prior_probability(interpretations, w_llm=0.5, w_bscore =0.7)
    interpretations = calculate_fit_quality(interpretations,w_rwp=1, w_score=0.5)
    interpretations = calculate_posterior_probability_of_interpretation(interpretations)
    json_file[sample] = interpretations

save_json("Data/prompt2/interpret_results1.json",json_file)

KeyError: 'LLM_interpretation_likelihood_llama'